In [1]:
import yfinance as yf
import pandas as pd
from talib import BBANDS, RSI, ATR, MFI

# Define the ticker symbols
tickerSymbols = ['NFLX', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM', 'V', 'GOOG', 'AMZN', 'AAP', 'MSFT', 
                 'AAPL', 'TSLA', 'NVDA', 'PYPL', 'INTC', 'CMCSA', 'CSCO', 'NFLX', 'PEP', 'ADBE', 'AMGN', 'AVGO', 'TXN', 'QCOM', 'COST', 
                 'SBUX', 'AMAT', 'BKNG', 'ADP', 'GILD', 'MDLZ', 'MRNA', 'VRSK', 'AAL', 'MELI', 'ZM', 'JD', 'ATVI', 'CSX', 'LRCX', 'INTU', 
                 'CTSH', 'ISRG', 'REGN', 'TMUS']

# Set parameters for the trading strategy
buy_rsi_threshold = 20
sell_rsi_threshold = 80
sell_mfi_threshold = 57
stop_loss_pct = 0.15

ticker_trade_metrics = []

# Iterate over each ticker
for tickerSymbol in tickerSymbols:
    # Get historical data for the ticker
    tickerData = yf.Ticker(tickerSymbol)
    df = tickerData.history(period='60d', interval='5m')
    
    # Calculate Bollinger Bands
    df['upper_band'], df['middle_band'], df['lower_band'] = BBANDS(df['Close'], timeperiod=20)

    # Calculate RSI
    df['rsi'] = RSI(df['Close'], timeperiod=14)

    # Calculate Keltner Channels
    df['keltner_middle'] = df['Close'].rolling(window=20).mean()
    df['atr'] = ATR(df['High'], df['Low'], df['Close'], timeperiod=20)
    df['keltner_upper'] = df['keltner_middle'] + (df['atr'] * 1.5)
    df['keltner_lower'] = df['keltner_middle'] - (df['atr'] * 1.5)

    # Calculate Money Flow Index
    df['mfi'] = MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)

    # Initialize variables
    position = 0
    entry_price = 0
    balance = 50000
    risk_amount = (.40 * balance) # 40% of starting balance
    # Create trade journal DataFrame for the ticker
    trade_journal = pd.DataFrame(columns=['Date', 'Action', 'Price', 'Shares', 'Balance', 'Ticker'])

    # Iterate over the data
    for i, row in df.iterrows():
        # Check if RSI is below the buy threshold and close price is below the lower Keltner Channel or lower Bollinger Band
        if row['rsi'] < buy_rsi_threshold and (row['Close'] < row['keltner_lower'] or row['Close'] < row['lower_band']):
            if position == 0:
                # Calculate the number of shares we can buy based on risk amount and current price
                num_shares = int(risk_amount / row['Close'])
                # Deduct the cost of the shares from our balance
                balance -= num_shares * row['Close']
                # Update our position
                position += num_shares
                # Set the entry price
                entry_price = row['Close']
                # Add the trade to the trade journal
                trade_journal = trade_journal.append({'Date': i, 'Action': 'Buy', 'Price': row['Close'], 'Shares': num_shares, 'Balance': balance, 'Ticker': tickerSymbol}, ignore_index=True)
        # Check if RSI is above the sell threshold and close price is above the upper Keltner Channel or upper Bollinger Band, or if price is below the stop loss level
        elif ((row['rsi'] > sell_rsi_threshold and (row['Close'] > row['keltner_upper'] or row['Close'] > row['upper_band'])) or (entry_price > 0 and row['Close'] < entry_price * (1 - stop_loss_pct)) or (row['mfi'] < sell_mfi_threshold and row['rsi'] > sell_rsi_threshold)):
            if position > 0:
                # Add the proceeds to our balance
                balance += position * row['Close']
                # Update our position
                position -= position
                # Add the trade to the trade journal
                trade_journal = trade_journal.append({'Date': i, 'Action': 'Sell', 'Price': row['Close'], 'Shares': position, 'Balance': balance, 'Ticker': tickerSymbol}, ignore_index=True)

    # Ensure that the trade journal has both 'Buy' and 'Sell' transactions before calculating metrics
    if 'Buy' in trade_journal['Action'].values and 'Sell' in trade_journal['Action'].values:
        buy_df = trade_journal[trade_journal["Action"] == "Buy"].copy().reset_index(drop=True)
        sell_df = trade_journal[trade_journal["Action"] == "Sell"].copy().reset_index(drop=True)
        
        if len(buy_df) != len(sell_df):
            min_len = min(len(buy_df), len(sell_df))
            buy_df = buy_df[:min_len]
            sell_df = sell_df[:min_len]

        num_wins = (sell_df["Price"] > buy_df["Price"]).sum()
        win_ratio = num_wins / len(buy_df)
        
        ticker_trade_metrics.append({
            'Ticker': tickerSymbol,
            'Win Ratio': win_ratio,
            'Number of Trades': len(buy_df),
        })

# Create a DataFrame from the trade metrics
trade_metrics_df = pd.DataFrame(ticker_trade_metrics)

# Sort recommendations by Win Ratio in descending order
recommendations = trade_metrics_df.sort_values(by='Win Ratio', ascending=False)

print("Recommended Stocks:")
print(recommendations)


Recommended Stocks:
   Ticker  Win Ratio  Number of Trades
0    NFLX   1.000000                 5
8     JPM   1.000000                 4
1    AAPL   1.000000                 5
14   AAPL   1.000000                 5
13   MSFT   1.000000                 5
42    CSX   1.000000                 4
21   NFLX   1.000000                 5
46   ISRG   1.000000                 3
36   VRSK   1.000000                 3
5    META   1.000000                 3
32    ADP   1.000000                 4
2    MSFT   1.000000                 5
43   LRCX   1.000000                 4
3   GOOGL   0.888889                 9
10   GOOG   0.875000                 8
23   ADBE   0.857143                 7
37    AAL   0.833333                 6
25   AVGO   0.833333                 6
33   GILD   0.833333                 6
16   NVDA   0.833333                 6
15   TSLA   0.833333                 6
7    NVDA   0.833333                 6
6    TSLA   0.833333                 6
19  CMCSA   0.800000                 5
20   